In [ ]:
"""
    Main training workflow
"""
from __future__ import division

import argparse
import glob
import os
import random
import signal
import time
import numpy as np

import torch
from pytorch_pretrained_bert import BertConfig


import distributed
from models import data_loader, model_builder
from models.data_loader import load_dataset
from models.model_builder import Summarizer
from tensorboardX import SummaryWriter
from models.reporter import ReportMgr
from models.stats import Statistics
from others.logging import logger
# from models.trainer import build_trainer
# build_trainer의 dependency package pyrouge.utils가 import되지 않아 직접 셀에 삽입
from others.logging import logger, init_logger
import easydict

args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'summary',
    "bert_data_path":'../bert_sample/korean',
    "model_path":'../models/bert_classifier',
    "bert_model":'../001_bert_morp_pytorch',
    "result_path":'../results/korean',
    "temp_dir":'.',
    "bert_config_path":'../001_bert_morp_pytorch/bert_config.json',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'0',
    "gpu_ranks":'0',
    "log_file":'../logs/bert_classifier',
    "test_from":'../models/bert_classifier2/model_step_35000.pt'
})


def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    #print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        #logger.info('* number of parameters: %d' % n_params)

    return trainer

class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summary(self, test_iter, step, cal_lead=False, cal_oracle=False):
        """ Validate model.
            valid_iter: validate data iterator
        Returns:
            :obj:`nmt.Statistics`: validation loss statistics
        """
        # Set model in validating mode.
        def _get_ngrams(n, text):
            ngram_set = set()
            text_length = len(text)
            max_index_ngram_start = text_length - n
            for i in range(max_index_ngram_start + 1):
                ngram_set.add(tuple(text[i:i + n]))
            return ngram_set

        def _block_tri(c, p):
            tri_c = _get_ngrams(3, c.split())
            for s in p:
                tri_s = _get_ngrams(3, s.split())
                if len(tri_c.intersection(tri_s))>0:
                    return True
            return False

        if (not cal_lead and not cal_oracle):
            self.model.eval()
        stats = Statistics()

        
        with torch.no_grad():
            for batch in test_iter:
                src = batch.src
                labels = batch.labels
                segs = batch.segs
                clss = batch.clss
                mask = batch.mask
                mask_cls = batch.mask_cls


                gold = []
                pred = []

                if (cal_lead):
                    selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
                elif (cal_oracle):
                    selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                    range(batch.batch_size)]
                else:
                    sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

                    # loss = self.loss(sent_scores, labels.float())
                    # loss = (loss * mask.float()).sum()
                    # batch_stats = Statistics(float(loss.cpu().data.numpy()), len(labels))
                    # stats.update(batch_stats)

                    sent_scores = sent_scores + mask.float()
                    sent_scores = sent_scores.cpu().data.numpy()
                    selected_ids = np.argsort(-sent_scores, 1)
                # selected_ids = np.sort(selected_ids,1)
                

        return selected_ids


    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        #logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)

            
def summary(args, b_list, device_id, pt, step, model):
    device_id = 0
    device = "cpu" if args.visible_gpus == '-1' else "cuda"
    if (pt != ''):
        test_from = pt
    else:
        test_from = args.test_from
    
    opt = vars(checkpoint['opt'])
    for k in opt.keys():
        if (k in model_flags):
            setattr(args, k, opt[k])
    #print(args)

    #model.load_cp(checkpoint)
    #model.eval()

    test_iter =data_loader.Dataloader(args, _lazy_dataset_loader(b_list),
                                  args.batch_size, device,
                                  shuffle=False, is_test=True)
    trainer = build_trainer(args, device_id, model, None)
    result = trainer.summary(test_iter,step)
    return result

def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

init_logger(args.log_file)
device = "cpu" if args.visible_gpus == '-1' else "cuda"
device_id = 0 if device == "cuda" else -1
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

import argparse
import json
import os
import time
import urllib3
from glob import glob
import collections
import six
import gc
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.utils import download as _download

def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    requestJson = {  
        "argument": {
            "text": text,
            "analysis_code": "morp"
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  openapi_key},
        body=json.dumps(requestJson)
    )

    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        json_sentence = json_return_obj["sentence"]
        for json_morp in json_sentence:
            for morp in json_morp["morp"]:
                return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

        return return_result

def get_kobert_vocab(cachedir="./tmp/"):
    # Add BOS,EOS vocab
    tokenizer = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/tokenizers/kobert_news_wiki_ko_cased-1087f8699e.spiece",
        #"fname": "/opt/ml/SumAI/bertsum/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece",
        "chksum": "ae5711deb3",
    }
    
    vocab_info = tokenizer
    vocab_file = _download(
        #vocab_info["url"], vocab_info["fname"], vocab_info["chksum"], cachedir=cachedir
        vocab_info["url"], vocab_info["chksum"], cachedir=cachedir
    )

    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
        vocab_file[0], padding_token="[PAD]", bos_token="[BOS]", eos_token="[EOS]"
    )
    return vocab_b_obj

    
class BertData():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        #self.tokenizer = Tokenizer(vocab_file_path)
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [''.join(s) for s in src]


        idxs = [i for i, s in enumerate(src) if (len(s) > 0)]

        src = [src[i][:20000] for i in idxs]
        src = src[:10000]

        if (len(src) < 3):
            return None

        src_txt = [''.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = text.split(' ')
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        tgt_txt = None
        src_txt = [original_src_txt[i] for i in idxs]
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt
    
def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")
        
class Tokenizer(object):
    
    def __init__(self, vocab_file_path):
        self.vocab_file_path = vocab_file_path
        """Loads a vocabulary file into a dictionary."""
        vocab = collections.OrderedDict()
        index = 0
        with open(self.vocab_file_path, "r", encoding='utf-8') as reader:

            while True:
                token = convert_to_unicode(reader.readline())
                if not token:
                    break

          ### joonho.lim @ 2019-03-15
                if token.find('n_iters=') == 0 or token.find('max_length=') == 0 :

                    continue
                token = token.split('\t')[0].strip('_')

                token = token.strip()
                vocab[token] = index
                index += 1
        self.vocab = vocab
        
    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            try:
                ids.append(self.vocab[token])
            except:
                ids.append(1)
        if len(ids) > 10000:
            raise ValueError(
                "Token indices sequence length is longer than the specified maximum "
                " sequence length for this BERT model ({} > {}). Running this"
                " sequence through BERT will result in indexing errors".format(len(ids), 10000)
            )
        return ids


def _lazy_dataset_loader(pt_file):
    dataset = pt_file    
    yield dataset
    
def News_to_input(text, openapi_key, tokenizer):
    newstemp = do_lang(openapi_key, text)
    news = newstemp.split(' ./SF ')[:-1]
    
    bertdata = BertData(tokenizer)
    sent_labels = [0] * len(news)
    tmp = bertdata.preprocess(news)
    if(not tmp): return None
    #print(tmp)
    b_data_dict = {"src":tmp[0],
               "tgt": [0],
               "labels":[0,0,0],
               "src_sent_labels":sent_labels,
               "segs":tmp[2],
               "clss":tmp[3],
               "src_txt":tmp[4],
               "tgt_txt":'hehe'}
    b_list = []
    b_list.append(b_data_dict) 
    return b_list

openapi_key = '9318dc23-24ac-4b59-a99e-a29ec170bf02'


import re
from nltk import word_tokenize, sent_tokenize
def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+[^~][^%](?:\.[-_0-9a-z]+)+[^%]', flags=re.IGNORECASE)
    pattern_others = re.compile(r'(\[)*(^\])*(◼|기자|뉴스|사진|자료|자료사진|출처|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|경제|한겨례|일보|미디어|데일리|한겨례|타임즈|위키트리|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|댓글|좋아요|공유하기|글씨 크게 보기|글씨 작게 보기|고화질|표준화질|자동 재생|키보드 컨트롤 안내|동영상 시작)(^\])*?(\])*( 제공)*')
    pattern_tag = re.compile(r'\<[^>][^가-힣 ]*?>')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('', result)
    result = pattern_tag.sub('', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()
    return result


def text_filter(title, text): # str -> 전처리 -> 문장 배열
    text = clean_byline(text)
    result = text.strip()
    sentences = sent_tokenize(result) 
    return_sentences = []
    for sent in sentences:
        for s in sent.split('\n'):
            s = s.strip()
            if(len(s)>12 and '다' in s[-3:]): 
                if(s[-1] != '.'):
                    s += '.'
                return_sentences.append(s+' ')
                
    s1 = set(title.split())
    s2 = set(return_sentences[0].split())
    actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
    if(actual_jaccard > 0.5):
        return_sentences = return_sentences[1:]
        
    return_sentences = return_sentences[:30]
    
    return return_sentences

/opt/conda/lib/python3.8/site-packages/mxnet/numpy/utils.py:36: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
  bool = onp.bool


AttributeError: module 'numpy' has no attribute 'bool'

In [ ]:
openapi_key = '9318dc23-24ac-4b59-a99e-a29ec170bf02'
chat_history = "뉴스 "
user_input = '''
일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다.
정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다.
앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다.
한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다.
가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다.
그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다.
가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다.
그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다.
'''
vocab = get_kobert_vocab()
tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)


bot_input_ids = News_to_input(chat_history + user_input, openapi_key, tokenizer)
print('------------------------------------------------------------')


#logger.info('Loading checkpoint from %s' % test_from)
checkpoint = torch.load('../models/bert_classifier/model_step_1000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
model.load_cp(checkpoint)
model.eval()  

chat_history_ids = summary(args, bot_input_ids, 0, '', None, model)
print(chat_history_ids)
print('------------------------------------------------------------')
print(chat_history_ids[0])
pred_lst = list(chat_history_ids[0][:3])
print(pred_lst)
print('------------------------------------------------------------')
final_text = ''
for i,a in enumerate(user_input.split('. ')):
    if i in pred_lst:
        print(i, a)
        final_text = final_text+a+'. '
chat_history = user_input + '<token>' +final_text
print('------------------------------------------------------------')
print(final_text)
print('------------------------------------------------------------')

In [49]:
from tqdm import tqdm 

checkpoint = torch.load('../models/bert_classifier2/model_step_35000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
model.load_cp(checkpoint)
model.eval()  
vocab = get_kobert_vocab()
tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)

def get_top_sentences(user_input, model, tokenizer):
    bot_input_ids = News_to_input(user_input, openapi_key, tokenizer)
    
    chat_history_ids = summary(args, bot_input_ids, 0, '', None, model)
    pred_lst = list(chat_history_ids[0])
    final_text = []
    for p in pred_lst:
        if(p < len(user_input.split('. ')) and len(user_input.split('. ')[p]) > 10):
            final_text.append((p, user_input.split('. ')[p]+'. '))
    return final_text


def add_topk_to_df(df, model, tokenizer):
    
    start = time.time()
    topk = []
    for i,context2 in enumerate(tqdm(df['context2'])):
        context2 = eval(context2)
        context = []
        for s in context2:
            if(len(s) > 600):
                s = s.split('.')[0]+' '
            if(len(s) < 600):
                context.append(s)
        
        context = context[:30]
        top = None
        if(len(context) > 4):
            top = get_top_sentences(' '.join(context), model, tokenizer)
        if(top):
            topk.append(top)
        else:
            topk.append('Hello world')
            
    df['topk'] = topk
    df = df.drop(df[df['topk'] == 'Hello world'].index)
    print(f"{time.time()-start:.4f} sec")
    return df

[2023-01-23 13:29:46,810 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-23 13:29:46,812 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [40]:
def add_context_to_df(df):
    contexts = []
    for i, cont in enumerate(tqdm(df['raw'])):
        if(cont):
            contexts.append(text_filter(cont))
        else:
            contexts.append('delete this')
    df['context2'] = contexts
    df = df.drop(df[df['context2'] == 'delete this'].index)
    return df

In [64]:
raw = df.raw[56]
for i,v in enumerate(text_filter(raw)):
    print(i,v)
    
context2 = df.context2[56]
context2 = eval(context2)
context = []
for s in context2:
    if(len(s) > 600):
        s = s.split('.')[0]+' '
    if(len(s) < 600):
        context.append(s)

context = context[:30]
context

0 국내 지방자치단체들이 테슬라의 생산기지인 '기가팩토리' 유치전에 본격적으로 뛰어들었다.일론 머스크 테슬라 최고경영자(CEO)가 지난달 23일 윤석열 대통령과 영상면담에서 "한국은 아시아권 최우선 투자 후보지 중 하나"라고 밝히면서 테슬라의 아시아 2공장 유력 후보지로 한국이 부상했기 때문이다. 
1 정부도 산업통상자원부와 대한무역투자진흥공사(KOTRA)를 중심으로 전담 팀을 구성해 테슬라의 국내 투자를 유치할 계획인 만큼 지자체들 움직임도 바빠졌다. 
2 1일 각 지자체에 따르면 현재 경북 포항시, 경기 고양시, 강원도 등이 테슬라의 기가팩토리 유치에 가장 적극적이다. 
3 이들 지자체는 산업부에 유치 의향을 전달하는 등 투자 강점을 알리는 데 집중하고 있다.가장 먼저 움직인 곳이 경북 포항시다. 
4 이철우 경북도지사와 이강덕 포항시장은 머스크와 윤 대통령의 영상면담 직후부터 기가팩토리 포항 유치를 위한 전략 마련에 돌입했다. 
5 이를 위해 포항시는 테슬라 유치팀을 구성하고 지난달 30일 산업부를 찾아 투자 강점 등을 담은 유치 의향서를 전달했다. 
6 포항시는 포스코의 철판 공급망과 함께 배터리 리사이클링 규제자유특구 등 2차전지 생태계 구축 등을 강점으로 내세우고 있다. 
7 또 포스텍의 연구 기반 시설과 영일만 배후산단 등 물류 기반 시설이 우수하다는 점도 강조하고 있다. 
8 포항시 관계자는 "테슬라 기가팩토리가 유치되면 1973년 포항제철소 1기 준공 이후 지역  발전의 최대 전환점이 될 것"이라며 "포항의 뛰어난 입지 여건을 기반으로 유치에 총력을 기울이겠다"고 말했다.경기 고양시도 정부와 경기도에 고양자유구역과 연계한 테슬라 기가팩토리 유치를 제안할 계획이다. 
9 고양시는 최근 경기도 자유구역 후보지로 선정된 점을 강점으로 내세우고 있다. 
10 자유구역으로 최종 지정될 경우 해외 투자자본 유치를 위한 세제 감면이나 규제 완화 등의 혜택이 부여된다. 
11 테슬라처럼 기술력을 갖춘 해외 기업이 입주할 만한 장점이 있는 셈이다. 
12 이동

['국내 지방자치단체들이 테슬라의 생산기지인 \'기가팩토리\' 유치전에 본격적으로 뛰어들었다.일론 머스크 테슬라 최고경영자(CEO)가 지난달 23일 윤석열 대통령과 영상면담에서 "한국은 아시아권 최우선 투자 후보지 중 하나"라고 밝히면서 테슬라의 아시아 2공장 유력 후보지로 한국이 부상했기 때문이다. ',
 '정부도 산업통상자원부와 대한무역투자진흥공사(KOTRA)를 중심으로 전담 팀을 구성해 테슬라의 국내 투자를 유치할 계획인 만큼 지자체들 움직임도 바빠졌다. ',
 '1일 각 지자체에 따르면 현재 경북 포항시, 경기 고양시, 강원도 등이 테슬라의 기가팩토리 유치에 가장 적극적이다. ',
 '이들 지자체는 산업부에 유치 의향을 전달하는 등 투자 강점을 알리는 데 집중하고 있다.가장 먼저 움직인 곳이 경북 포항시다. ',
 '이철우 경북도지사와 이강덕 포항시장은 머스크와 윤 대통령의 영상면담 직후부터 기가팩토리 포항 유치를 위한 전략 마련에 돌입했다. ',
 '이를 위해 포항시는 테슬라 유치팀을 구성하고 지난달 30일 산업부를 찾아 투자 강점 등을 담은 유치 의향서를 전달했다. ',
 '포항시는 포스코의 철판 공급망과 함께 배터리 리사이클링 규제자유특구 등 2차전지 생태계 구축 등을 강점으로 내세우고 있다. ',
 '또 포스텍의 연구 기반 시설과 영일만 배후산단 등 물류 기반 시설이 우수하다는 점도 강조하고 있다. ',
 '포항시 관계자는 "테슬라 기가팩토리가 유치되면 1973년 포항제철소 1기 준공 이후 지역  발전의 최대 전환점이 될 것"이라며 "포항의 뛰어난 입지 여건을 기반으로 유치에 총력을 기울이겠다"고 말했다.경기 고양시도 정부와 경기도에 고양자유구역과 연계한 테슬라 기가팩토리 유치를 제안할 계획이다. ',
 '고양시는 최근 경기도 자유구역 후보지로 선정된 점을 강점으로 내세우고 있다. ',
 '자유구역으로 최종 지정될 경우 해외 투자자본 유치를 위한 세제 감면이나 규제 완화 등의 혜택이 부여된다. ',
 '테슬라처럼 기술력을 갖춘 해외 기업이 입주할

In [66]:
' '.join(context)

'국내 지방자치단체들이 테슬라의 생산기지인 \'기가팩토리\' 유치전에 본격적으로 뛰어들었다.일론 머스크 테슬라 최고경영자(CEO)가 지난달 23일 윤석열 대통령과 영상면담에서 "한국은 아시아권 최우선 투자 후보지 중 하나"라고 밝히면서 테슬라의 아시아 2공장 유력 후보지로 한국이 부상했기 때문이다.  정부도 산업통상자원부와 대한무역투자진흥공사(KOTRA)를 중심으로 전담 팀을 구성해 테슬라의 국내 투자를 유치할 계획인 만큼 지자체들 움직임도 바빠졌다.  1일 각 지자체에 따르면 현재 경북 포항시, 경기 고양시, 강원도 등이 테슬라의 기가팩토리 유치에 가장 적극적이다.  이들 지자체는 산업부에 유치 의향을 전달하는 등 투자 강점을 알리는 데 집중하고 있다.가장 먼저 움직인 곳이 경북 포항시다.  이철우 경북도지사와 이강덕 포항시장은 머스크와 윤 대통령의 영상면담 직후부터 기가팩토리 포항 유치를 위한 전략 마련에 돌입했다.  이를 위해 포항시는 테슬라 유치팀을 구성하고 지난달 30일 산업부를 찾아 투자 강점 등을 담은 유치 의향서를 전달했다.  포항시는 포스코의 철판 공급망과 함께 배터리 리사이클링 규제자유특구 등 2차전지 생태계 구축 등을 강점으로 내세우고 있다.  또 포스텍의 연구 기반 시설과 영일만 배후산단 등 물류 기반 시설이 우수하다는 점도 강조하고 있다.  포항시 관계자는 "테슬라 기가팩토리가 유치되면 1973년 포항제철소 1기 준공 이후 지역  발전의 최대 전환점이 될 것"이라며 "포항의 뛰어난 입지 여건을 기반으로 유치에 총력을 기울이겠다"고 말했다.경기 고양시도 정부와 경기도에 고양자유구역과 연계한 테슬라 기가팩토리 유치를 제안할 계획이다.  고양시는 최근 경기도 자유구역 후보지로 선정된 점을 강점으로 내세우고 있다.  자유구역으로 최종 지정될 경우 해외 투자자본 유치를 위한 세제 감면이나 규제 완화 등의 혜택이 부여된다.  테슬라처럼 기술력을 갖춘 해외 기업이 입주할 만한 장점이 있는 셈이다.  이동환 고양시장은 "자유구역으로 지정되고 적절한 세

In [71]:
a = get_top_sentences(' '.join(context[1:20]), model, tokenizer)
a

TypeError: 'NoneType' object is not iterable

In [42]:
pattern_others = re.compile(r'(\[)*(^\])*(◼|기자|뉴스|사진|자료|자료사진|출처|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|경제|한겨례|일보|미디어|데일리|한겨례|타임즈|위키트리|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|댓글|좋아요|공유하기|글씨 크게 보기|글씨 작게 보기|고화질|표준화질|자동 재생|키보드 컨트롤 안내|동영상 시작)(^\])*?(\])*( 제공)*')

result = '[우성덕 기자 / 이상헌 기자]'
result = pattern_others.sub('', result)
result

'[우성덕   이상헌 '

In [48]:
for raw in eval(df.context2[0]):
    print(raw)

윤석열 대통령은 15일 국정과제 점검회의에서 규제 완화와 다주택자 세 중과 해제 검토,주택공급, 공공·민간 믹스 등 3대개혁 청을 제시했다. 
윤 대통령은 이날 오후 2시부터 청와대 영빈관에서 열린 제1차 국정과제 점검회의에서 국민 패널 2명으로부터 ‘내 집 마련’과 ‘부동산 경기 활성화’ 방안과 관련한 질문을 받고 부동산 시장 연착륙을 위한 규제 완화 필요성을 강조했다. 
전 정부에서 강화된 부동산 규제를 일제히 풀 경우 부작용을 우려해 그간 속도 조절을 해왔지만, 고금리 등으로 상황이 급변한 시장 대응을 위해 다시 규제 완화 고삐를 죄겠다고 했다.이같은 발언은 규제완화를 통한 부동산 시장을 활성화시키겠다는 것으로 풀이된다. 
윤 대통령은 “제가 정부를 맡기 전까지 공급 측면과 수요 측면의 불합리한 복합 규제 때문에 집값이 너무 천정부지로 솟고 거래물량이 위축됐다”며 “시장 정상화가 중요하다고 해서 많은 규제를 풀고 정상화하려고 했는데, 고금리 상황 때문에 다시 부동산 가격이 하락하는 추세를 보였다”고 진단했다. 
그러면서 “(전 정부의) 잘못된 정책으로 인한 현상이라고 해도, 그것을 일시에 제거하다 보면 시장에 혼란이 일어나서 결국 국민에게 불편을 줄 수 있어서 시장 정상화의 속도를 조율해야 한다고 생각했다”며 수요 규제를 조금 더 빠른 속도로 풀어나가서 시장이 안정 찾는데 최선의 노력할 것이라고 강조했다. 
이 같은 발언은 최근 고금리와 집값 하락 우려로 거래가 메마르면서 곳곳에서 ‘경고음’이 터지는 데 따른 것으로 풀이된다. 
내놓은 매물이 나가지 않아 이사를 못가는 상황이 발생하거나, ‘영끌’ 매입자들의 금리 부담 급증, 집값 하락세에 따른 보유세 부담 호소, 빠른 월세화 현상 등이다. 
종합부동산세의 경우 국회에서 관련법 개정 논의가 이어지고 있다. 
여야는 앞서 종부세 중과세율(1.2~6.0%)을 적용하는 조정대상지역 2주택 이상 소유자에 대해 중과를 폐지하고 일반세율(0.5~2.7%)을 적용하는 방안 등을 담은 잠정 합의안을 도출한 상태다. 
종

In [44]:
all_paths = [
 'bertopic(mpnet)',
 'bertopic(paraphrase)',
 'kmeans_12',
 'hdbscan',
 'kmeans_8',
 'bertopic(SR-BERT)',
 'kmeans_4']

import os
from os import listdir
from os.path import isfile, join, isdir
import pandas as pd

i = 0
mypath = f'./{all_paths[i]}'
print(mypath)
onlyfiles = [f for f in listdir(mypath+'/raw') if isfile(join(mypath+'/raw', f)) and f.split('.')[1] == 'csv']
print(onlyfiles)
############### raw 전처리 ################
for i,fname in enumerate(onlyfiles):
    file = fname.split('_crwal')[0]
    filepath = f'{mypath}/raw/{onlyfiles[i]}'
    df = pd.read_csv(filepath)
    print(i, len(df))
    new_df = add_context_to_df(df)
    if not os.path.exists(f'./preprocessed/{mypath}/'):
        os.makedirs(f'./preprocessed/{mypath}/')
    new_df.to_csv(f'./preprocessed/{mypath}/{file}_pre.csv')

############### 추출요약 ################
for i,fname in enumerate(onlyfiles):
    file = fname.split('_crwal')[0]
    filepath = f'./preprocessed/{mypath}/{file}_pre.csv'
    df = pd.read_csv(filepath)
    print(i, len(df))
    new_df = add_topk_to_df(df, model, tokenizer)
    if not os.path.exists(f'./preprocessed2/{mypath}/'):
        os.makedirs(f'./preprocessed2/{mypath}/')
    new_df.to_csv(f'./preprocessed2/{mypath}/{file}_pre2.csv')

./bertopic(mpnet)
['bertopic_윤석열_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221215_crwal_news_context_raw.csv', 'bertopic_윤석열_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_윤석열_20221201_20221215_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221215_crwal_news_context_raw.csv']
0 88


100%|██████████| 88/88 [00:00<00:00, 827.36it/s]


1 16


100%|██████████| 16/16 [00:00<00:00, 673.09it/s]


2 128


...

8 122


100%|██████████| 122/122 [00:00<00:00, 801.81it/s]


0 88


  0%|          | 0/88 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not iterable

In [32]:
all_paths = [
 'bertopic(mpnet)',
 'bertopic(paraphrase)',
 'kmeans_12',
 'hdbscan',
 'kmeans_8',
 'bertopic(SR-BERT)',
 'kmeans_4']

import os
from os import listdir
from os.path import isfile, join, isdir
import pandas as pd

for i in range(len(all_paths)):
    if i == 0:
        continue
    mypath = f'./{all_paths[i]}'
    print(mypath)
    onlyfiles = [f for f in listdir(mypath+'/raw') if isfile(join(mypath+'/raw', f)) and f.split('.')[1] == 'csv']
    print(onlyfiles)
    ############### raw 전처리 ################
    for i,fname in enumerate(onlyfiles):
        file = fname.split('_crwal')[0]
        filepath = f'{mypath}/raw/{onlyfiles[i]}'
        df = pd.read_csv(filepath)
        print(i, len(df))
        new_df = add_context_to_df(df)
        if not os.path.exists(f'./preprocessed/{mypath}/'):
            os.makedirs(f'./preprocessed/{mypath}/')
        new_df.to_csv(f'./preprocessed/{mypath}/{file}_pre.csv')

    ############### 추출요약 ################
    for i,fname in enumerate(onlyfiles):
        file = fname.split('_crwal')[0]
        filepath = f'./preprocessed/{mypath}/{file}_pre.csv'
        df = pd.read_csv(filepath)
        print(i, len(df))
        new_df = add_topk_to_df(df, model, tokenizer)
        if not os.path.exists(f'./preprocessed2/{mypath}/'):
            os.makedirs(f'./preprocessed2/{mypath}/')
        new_df.to_csv(f'./preprocessed2/{mypath}/{file}_pre2.csv')

./bertopic(paraphrase)
['bertopic_윤석열_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221215_crwal_news_context_raw.csv', 'bertopic_윤석열_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_윤석열_20221201_20221215_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221215_crwal_news_context_raw.csv']
0 111


100%|██████████| 111/111 [00:00<00:00, 798.41it/s]


1 16


100%|██████████| 16/16 [00:00<00:00, 651.68it/s]


2 126


...

8 134


100%|██████████| 134/134 [00:00<00:00, 714.65it/s]


0 111


  0%|          | 0/111 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not iterable

In [299]:
z = 'bertopic_부동산_20221201_20221231_pre.csv'
df = pd.read_csv(f'./preprocessed/bertopic(mpnet)/{z}')
len(df)

318

In [ ]:
temp = f'./preprocessed/bertopic(mpnet)/{file}_pre.csv'

In [343]:
text = '''
['한종희 삼성전자 부회장이 지난 1월 4일 미국 라스베이거스 베네시안 팔라조에서 ‘미래를 위한 동행(Together for tomorrow)’을 주제로 CES 2022 기조 연설을 하고 있다. ', '지난 5일(현지시간) 미국 네바다주 라스베이거스컨벤션센터에서 CES 2022 관람객과 취재진이 전시장 입장을 기다리고 있다. ', '한종희 삼성전자 디바이스경험(DX)(부회장)은 다음 달 초 미국 라스베이거스에서 열리는 세계 최대 가전·IT 전시회인 ‘CES 2023’에서 ‘맞춤형 경험으로 여는 초(超)연결 시대’(Bringing Calm to Our Connected World)를 제안할 예정이라고 15일 밝혔다 ', '올해 초 열린 ‘CES 2022’에서는 ‘미래를 위한 동행’을 주제로 기조연설을 했다 ', '삼성전자는 CES 2023에서 초연결 시대에 기반한 진화된 스마트싱스 경험을 선보이며, 지속가능한 미래를 위한 여정을 공유할 계획이다 ','한종희 삼성전자 부회장이 지난 1월 4일 미국 라스베이거스 베네시안 팔라조에서 ‘미래를 위한 동행(Together for tomorrow)’을 주제로 CES 2022 기조 연설을 하고 있다. ', '지난 5일(현지시간) 미국 네바다주 라스베이거스컨벤션센터에서 CES 2022 관람객과 취재진이 전시장 입장을 기다리고 있다. ', '한종희 삼성전자 디바이스경험(DX)(부회장)은 다음 달 초 미국 라스베이거스에서 열리는 세계 최대 가전·IT 전시회인 ‘CES 2023’에서 ‘맞춤형 경험으로 여는 초(超)연결 시대’(Bringing Calm to Our Connected World)를 제안할 예정이라고 15일 밝혔다 ', '올해 초 열린 ‘CES 2022’에서는 ‘미래를 위한 동행’을 주제로 기조연설을 했다 ', '삼성전자는 CES 2023에서 초연결 시대에 기반한 진화된 스마트싱스 경험을 선보이며, 지속가능한 미래를 위한 여정을 공유할 계획이다 ']
'''
context = eval(text)
top = get_top_sentences(' '.join(context), model, tokenizer)
top

[(2,
  ' 한종희 삼성전자 디바이스경험(DX)(부회장)은 다음 달 초 미국 라스베이거스에서 열리는 세계 최대 가전·IT 전시회인 ‘CES 2023’에서 ‘맞춤형 경험으로 여는 초(超)연결 시대’(Bringing Calm to Our Connected World)를 제안할 예정이라고 15일 밝혔다  올해 초 열린 ‘CES 2022’에서는 ‘미래를 위한 동행’을 주제로 기조연설을 했다  삼성전자는 CES 2023에서 초연결 시대에 기반한 진화된 스마트싱스 경험을 선보이며, 지속가능한 미래를 위한 여정을 공유할 계획이다  한종희 삼성전자 부회장이 지난 1월 4일 미국 라스베이거스 베네시안 팔라조에서 ‘미래를 위한 동행(Together for tomorrow)’을 주제로 CES 2022 기조 연설을 하고 있다. '),
 (3,
  ' 지난 5일(현지시간) 미국 네바다주 라스베이거스컨벤션센터에서 CES 2022 관람객과 취재진이 전시장 입장을 기다리고 있다. '),
 (0,
  '한종희 삼성전자 부회장이 지난 1월 4일 미국 라스베이거스 베네시안 팔라조에서 ‘미래를 위한 동행(Together for tomorrow)’을 주제로 CES 2022 기조 연설을 하고 있다. '),
 (1,
  ' 지난 5일(현지시간) 미국 네바다주 라스베이거스컨벤션센터에서 CES 2022 관람객과 취재진이 전시장 입장을 기다리고 있다. ')]

In [333]:
context2 = eval(df.context[76])
context2 = context2[:25]
context = []
for s in context2:
    if(len(s) > 300):
        s = s.split('.')[0]+' '
    if(len(s) > 300):
        s = s[:300]
    context.append(s)
get_top_sentences(' '.join(context), model, tokenizer)

[(4,
  '한 부회장은 다음달 4일 오후 2시 미국 서부시간 라스베이거스 만 달레이 베이 볼룸에서 열리는 삼성전자 프레스 컨퍼런스에 대표 연사로 나선다. '),
 (9,
  '한 부회장은 또 연결은 보다 쉬워지고, 개개인의 맞춤 경험은 AI로 더욱 정교해지며, 기 기간 연결은 보다 안전해질 것 이라며   삼성은 스마트싱스 가시화 를 위해 업계의 다양한 파트너들과 긴밀하게 협력하며, 개인의 취향과 라이프스타일에 최적화된 맞춤형 연결 경험을 보다 안전하고 풍부하게 제공할 수 있는 새로운 길 을 열어 보일 것 이라고 말했다. '),
 (8,
  '삼성전자는 CES 2023에서 초연결 시대에 기반 한 진화된 스마트 싱스 경험을 선보이며, 지속가능한 미래를 위한 여정을 공유할 계획이다. '),
 (3,
  '한 부회장은 이 날 삼성전자 뉴스룸에 올린 CES 2023 초연결 시대를 위한 혁신 제목의 기 고문을 통해 삼성전자는 이 번 CES 2023에서 캄테크 Calm Technology 철학을 바탕으로 한층 강화된 보안과 사물의 초연결 생태계에서 누리는 새롭고 확장된 스마트 싱스 SmartThin gs 경험을 선보인 다 고 밝혔다. '),
 (2,
  '라스 베이거스 연합뉴스 한종희 삼성전자 디바이스 경험 DX 부문장 부회장 은 다음 달 초 미국 라스베이거스에서 열리는 세계 최대 가전 IT 전시회인 CES 2023 에서 맞춤형 경험으로 여는 초 연결 시대 Bringing Calm to Our Con nected World 를 제안할 예정이라고 15일 밝혔다. '),
 (5, '올해 초 열린 CES 2022 에서는 미래를 위한 동행 을 주제로 기조연설을 했다. '),
 (6,
  '한 부회장은 기 고문에서 팬데믹과 글로벌 경제의 불확실성이 더해져 지금까지 겪어보지 못했던  새로운 패러다임을 맞이하고 있다 라면서 나와 내 가족이 살아가는 환경 과 경험 의 중요성이 한 층 더 커졌고, 미래 세대가 살아갈 지속가능한 미래에 대한 고민도 깊어졌다 고 현 상황을 진단했다. ')

In [11]:
import os
from os import listdir
from os.path import isfile, join, isdir
import pandas as pd

i = 0
mypath = f'./{all_paths[i]}'
onlyfiles = [f for f in listdir(mypath+'/raw') if isfile(join(mypath+'/raw', f)) and f.split('.')[1] == 'csv']

############### 추출요약 ################
for i,fname in enumerate(onlyfiles):
    if i > 1:
        file = fname.split('_crwal')[0]
        filepath = f'./preprocessed/{mypath}/{file}_pre.csv'
        df = pd.read_csv(filepath)
        print(i, len(df))
        new_df = add_topk_to_df(df, model, tokenizer)
        if not os.path.exists(f'./preprocessed2/{mypath}/'):
            os.makedirs(f'./preprocessed2/{mypath}/')
        new_df.to_csv(f'./preprocessed2/{mypath}/{file}_pre2.csv')

2 128


100%|██████████| 128/128 [00:44<00:00,  2.87it/s]


44.6000 sec
3 0


0it [00:00, ?it/s]


0.0035 sec
4 318


...

5.0490 sec
7 47


100%|██████████| 47/47 [00:21<00:00,  2.20it/s]


21.3915 sec
8 122


100%|██████████| 122/122 [00:51<00:00,  2.39it/s]

51.1139 sec


In [12]:
import os
from os import listdir
from os.path import isfile, join, isdir

for i in range(len(all_paths)):
    if i == 0:
        continue
    mypath = f'./{all_paths[i]}'
    print(mypath)
    onlyfiles = [f for f in listdir(mypath+'/raw') if isfile(join(mypath+'/raw', f)) and f.split('.')[1] == 'csv']
    print(onlyfiles)
    ############### raw 전처리 ################
    for i,fname in enumerate(onlyfiles):
        file = fname.split('_crwal')[0]
        filepath = f'{mypath}/raw/{onlyfiles[i]}'
        df = pd.read_csv(filepath)
        print(i, len(df))
        new_df = add_context_to_df(df)
        if not os.path.exists(f'./preprocessed/{mypath}/'):
            os.makedirs(f'./preprocessed/{mypath}/')
        new_df.to_csv(f'./preprocessed/{mypath}/{file}_pre.csv')

    ############### 추출요약 ################
    for i,fname in enumerate(onlyfiles):
        file = fname.split('_crwal')[0]
        filepath = f'./preprocessed/{mypath}/{file}_pre.csv'
        df = pd.read_csv(filepath)
        print(i, len(df))
        new_df = add_topk_to_df(df, model, tokenizer)
        if not os.path.exists(f'./preprocessed2/{mypath}/'):
            os.makedirs(f'./preprocessed2/{mypath}/')
        new_df.to_csv(f'./preprocessed2/{mypath}/{file}_pre2.csv')

./bertopic(paraphrase)
['bertopic_윤석열_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221215_crwal_news_context_raw.csv', 'bertopic_윤석열_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221231_crwal_news_context_raw.csv', 'bertopic_삼성전자_20221201_20221203_crwal_news_context_raw.csv', 'bertopic_윤석열_20221201_20221215_crwal_news_context_raw.csv', 'bertopic_부동산_20221201_20221215_crwal_news_context_raw.csv']
0 111


  0%|          | 0/111 [00:00<?, ?it/s]


NameError: name 'text_filter' is not defined

In [ ]:
for i,file in enumerate(onlyfiles):
    filepath = f'./preprocessed/{mypath}/{file}_pre.csv'
    df = pd.read_csv(filepath)
    print(i, len(df))
    new_df = add_topk_to_df(df, model, tokenizer)
    if not os.path.exists(f'./preprocessed2/{mypath}/'):
        os.makedirs(f'./preprocessed2/{mypath}/')
    new_df.to_csv(f'./preprocessed2/{mypath}/{file}_pre2.csv')

In [ ]:
for i,file in enumerate(onlyfiles):
    filepath = f'{mypath}/raw/{onlyfiles[i]}'
    df = pd.read_csv(filepath)
    print(i, len(df))
    new_df = add_context_to_df(df)
    if not os.path.exists(f'./preprocessed/{mypath}/'):
        os.makedirs(f'./preprocessed/{mypath}/')
    new_df.to_csv(f'./preprocessed/{mypath}/{file}_pre.csv')

In [240]:
text = "신혜진 기자 어쩌구저쩌구 (2%~3&) 뉴스 제목 [차민수 기자] <tag>오늘</tag>부터 <p>마스크 해제</p> 책 이름 <안녕하세요> 저는 신혜진입니다 [연합뉴스] 제공"
#text = "여야는 앞서 종부세 중과세율(1.2~6.0%)을 적용하는 조정대상지역 2주택 이상 소유자에 대해 중과를 폐지하고 일반세율(0.5~2.7%)을 적용하는 방안 등을 담은 잠정 합의안을 도출한 상태다. 종부세법 개정안은 예산안이 처리되는대로 국회 본회의에 오를 것으로 보인다. "
#text = '''서대구 역세권 사업 ‘공공주도로 개발’ 뉴스9(대구) 입력 2022.12.01 (21:47) 수정 2022.12.01 (22:05)\n\n댓글\n\n좋아요\n\n공유하기 글씨 크게보기\n\n가\n\n글씨 작게보기\n\n고화질 표준화질 자동재생 키보드 컨트롤 안내 동영상영역 시작 동영상 시작 동영상영역 끝 동영상 고정 취소 이전기사 이전기사 다음기사 다음기사\n\n고화질 표준화질 자동재생 키보드 컨트롤 안내 동영상영역 시작 동영상 시작 동영상영역 끝 동영상설명 동영상 고정 취소\n\n[앵커]\n\n\n\n대구시가 서대구 역세권 개발 방식을 민·관 공동추진에서 공공주도로 바꾸기로 했습니다.'''

text_filter(text)

['어쩌구저쩌구 (2%~3&)제목 오늘부터 마스크 해제 책 이름 <안녕하세요> 저는 신혜진입니다. ']

In [241]:
#pattern_others2 = re.compile(r'\<[^가-힣][^>]*?\>')
#pattern_others2 = re.compile(r'\[.[^]]*?\]')
pattern_others = re.compile(r'\[[^\]]*[기자|사진|자료|자료사진|출처|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지][^\]]*?\]')

re.findall(pattern_others, text)

['[차민수 기자]', '[연합뉴스]']

In [245]:
i = 1

path = 'hdbscan'
onlyfiles = [f for f in listdir(path+'/raw') if isfile(join(path+'/raw', f)) and f.split('.')[1] == 'csv']


filepath = f'{path}/raw/{onlyfiles[i]}'
df = pd.read_csv(filepath)
raw = df.raw[6]

In [294]:
onlyfiles[0].split('_raw')[0]

'bertopic_윤석열_20221201_20221231_crwal_news_context'

In [246]:
raw

'외지인 아파트 매입 두달새 5.8%P·1년새 17%P 하락\n\n서울거주자 춘천아파트 매입 한 달간 7건 사상 최저\n\n지방 부동산 활성화 위한 취득세 중과 해제 시급해져\n\n◇1일부터 시행되는 LTV·주택담보대출 완화 내용<제공=연합뉴스>\n\n수도권을 포함한 전국적인 부동산 규제지역 해제 이후 강원도내 아파트에 대한 외지인 투자가 급감한 것으로 나타났다.\n\n한국부동산원에 따르면 10월 기준 도내 아파트 매매거래 884건 중 외지인이 아파트를 매입한 거래는 245건, 27.7%로 조사됐다. 규제지역 해제 직전인 지난 8월에는 이 비율이 33.5%였다. 외지인 투자가 활발했던 지난해 10월 도내 아파트 매매거래 2,742건 중 외지인 매입 비율이 45.4%(1,246건)였던 점을 감안하면 1년 새 17.7%포인트나 하락한 셈이다.\n\n외지인들이 지난 8월까지 비규제지역인 강원도의 아파트를 활발하게 매입했지만 정부의 수도권 규제지역 해제가 본격화된 9월부터 그 수치가 뚝 떨어진 것.\n\n지역별로는 수도권과 가까운 춘천과 원주에서 외지인 매입 감소 폭이 컸다. 춘천지역 아파트를 서울 거주자가 산 경우는 지난해 10월 93건에서 올해 10월 7건으로 92.5% 급감했다. 춘천에서 월 단위로 서울 거주자 매입이 10건 이하를 기록한 것은 통계 작성이 시작된 2006년 1월 이후 사상 처음이다. 원주의 경우 같은 기간 서울 거주자 매입은 132건에서 10건으로 92.4%나 줄었다. 지난해 외지인 매입 비율이 전국 1위를 기록할 정도로 높았던 만큼 하락도 가팔랐다.\n\n토지 거래에서도 외지인 이탈이 두드러졌다. 지난 10월 강원지역 순수토지 거래(필지 수 기준) 중 외지인은 2,030건을 매입해 전년동월(3,532건) 대비 42.5% 감소했다. 같은 기간 도내 거주자가 매입한 건수는 15.4% 감소에 그쳤다.\n\n규제지역은 지난 9월부터 차례로 해제되는 추세로 최근에는 서울, 경기 일부를 제외한 모든 지역을 규제대상에서 해제했다. 더욱이 1일부터 15억원

In [247]:
text_filter(raw)

['수도권을 포함한 전국적인 부동산 규제지역 해제 이후 강원도내 아파트에 대한 외지인 투자가 급감한 것으로 나타났다. ',
 '한국부동산원에 따르면 10월 기준 도내 아파트 매매거래 884건 중 외지인이 아파트를 매입한 거래는 245건, 27.7%로 조사됐다. ',
 '규제지역 해제 직전인 지난 8월에는 이 비율이 33.5%였다. ',
 '외지인 투자가 활발했던 지난해 10월 도내 아파트 매매거래 2,742건 중 외지인 매입 비율이 45.4%(1,246건)였던 점을 감안하면 1년 새 17.7%포인트나 하락한 셈이다. ',
 '지역별로는 수도권과 가까운 춘천과 원주에서 외지인 매입 감소 폭이 컸다. ',
 '춘천지역 아파트를 서울 거주자가 산 경우는 지난해 10월 93건에서 올해 10월 7건으로 92.5% 급감했다. ',
 '춘천에서 월 단위로 서울 거주자 매입이 10건 이하를 기록한 것은 통계 작성이 시작된 2006년 1월 이후 사상 처음이다. ',
 '원주의 경우 같은 기간 서울 거주자 매입은 132건에서 10건으로 92.4%나 줄었다. ',
 '지난해 외지인 매입 비율이 전국 1위를 기록할 정도로 높았던 만큼 하락도 가팔랐다. ',
 '토지 거래에서도 외지인 이탈이 두드러졌다. ',
 '지난 10월 강원지역 순수토지 거래(필지 수 기준) 중 외지인은 2,030건을 매입해 전년동월(3,532건) 대비 42.5% 감소했다. ',
 '같은 기간 도내 거주자가 매입한 건수는 15.4% 감소에 그쳤다. ',
 '규제지역은 지난 9월부터 차례로 해제되는 추세로 최근에는 서울, 경기 일부를 제외한 모든 지역을 규제대상에서 해제했다. ',
 '더욱이 1일부터 15억원 초과 아파트에도 주택담보대출이 허용되는 등 추가 완화 방안이 본격화되면서 도내 외지인 매입 규모는 더욱 줄어들 전망이다. ',
 '이처럼 외지인들의 도내 아파트와 토지 매입이 줄어들 것으로 예상되자 지방 부동산의 취득세 중과 해제 등을 요구하는 목소리가 높아지고 있다. ',
 '강문식 한국공인중개사협회 춘천

In [165]:
text_filter(raw)

['페이지\n\n윤 대통령, 제1차 국정과제 점검회의 다주택자 세 중과 해제 검토 주택공급, 공공·민간 믹스…이달말 공공주택 50만가구 사전청약\n\n윤석열 대통령은 15일 국정과제 점검회의에서 규제 완화와 다주택자 세 중과 해제 검토,주택공급, 공공·민간 믹스 등 3대개혁 청사진을 제시했다.',
 '윤 대통령은 이날 오후 2시부터 청와대 영빈관에서 열린 제1차 국정과제 점검회의에서 국민 패널 2명으로부터 ‘내 집 마련’과 ‘부동산 경기 활성화’ 방안과 관련한 질문을 받고 부동산 시장 연착륙을 위한 규제 완화 필요성을 강조했다.',
 '전 정부에서 강화된 부동산 규제를 일제히 풀 경우 부작용을 우려해 그간 속도 조절을 해왔지만, 고금리 등으로 상황이 급변한 시장 대응을 위해 다시 규제 완화 고삐를 죄겠다고 했다.이같은 발언은 규제완화를 통한 부동산 시장을 활성화시키겠다는 것으로 풀이된다.',
 '불합리한 복합 규제 때문…규제 완화 ‘속도’\n\n윤 대통령은 “제가 정부를 맡기 전까지 공급 측면과 수요 측면의 불합리한 복합 규제 때문에 집값이 너무 천정부지로 솟고 거래물량이 위축됐다”며 “시장 정상화가 중요하다고 해서 많은 규제를 풀고 정상화하려고 했는데, 고금리 상황 때문에 다시 부동산 가격이 하락하는 추세를 보였다”고 진단했다.',
 '그러면서 “(전 정부의) 잘못된 정책으로 인한 현상이라고 해도, 그것을 일시에 제거하다 보면 시장에 혼란이 일어나서 결국 국민에게 불편을 줄 수 있어서 시장 정상화의 속도를 조율해야 한다고 생각했다”며 수요 규제를 조금 더 빠른 속도로 풀어나가서 시장이 안정 찾는데 최선의 노력할 것이라고 강조했다.',
 '이 같은 발언은 최근 고금리와 집값 하락 우려로 거래가 메마르면서 곳곳에서 ‘경고음’이 터지는 데 따른 것으로 풀이된다.',
 '내놓은 매물이 나가지 않아 이사를 못가는 상황이 발생하거나, ‘영끌’ 매입자들의 금리 부담 급증, 집값 하락세에 따른 보유세 부담 호소, 빠른 월세화 현상 등이다.',
 '다주택자 세 중과 해제 검

In [16]:
text_filter(raw)

['강해인 기자  기자페이지 윤 대통령, 제1차 국정과제 점검회의 다주택자 세 중과 해제 검토 주택공급, 공공 민간 믹스 이달말 공공주택 50만가구 사전청약 윤석열 대통령은 15일 국정과제 점검회의에서 규제 완화와 다주택자 세 중과 해제 검토,주택공급, 공공 민간 믹스 등 3대개혁 청사진을 제시했다.',
 '윤 대통령은 이날 오후 2시부터 청와대 영빈관에서 열린 제1차 국정과제 점검회의에서 국민 패널 2명으로부터  내 집 마련 과  부동산 경기 활성화  방안과 관련한 질문을 받고 부동산 시장 연착륙을 위한 규제 완화 필요성을 강조했다.',
 '전 정부에서 강화된 부동산 규제를 일제히 풀 경우 부작용을 우려해 그간 속도 조절을 해왔지만, 고금리 등으로 상황이 급변한 시장 대응을 위해 다시 규제 완화 고삐를 죄겠다고 했다.이같은 발언은 규제완화를 통한 부동산 시장을 활성화시키겠다는 것으로 풀이된다.',
 '불합리한 복합 규제 때문 규제 완화  속도 윤 대통령은  제가 정부를 맡기 전까지 공급 측면과 수요 측면의 불합리한 복합 규제 때문에 집값이 너무 천정부지로 솟고 거래물량이 위축됐다 며  시장 정상화가 중요하다고 해서 많은 규제를 풀고 정상화하려고 했는데, 고금리 상황 때문에 다시 부동산 가격이 하락하는 추세를 보였다 고 진단했다.',
 '그러면서  전 정부의  잘못된 정책으로 인한 현상이라고 해도, 그것을 일시에 제거하다 보면 시장에 혼란이 일어나서 결국 국민에게 불편을 줄 수 있어서 시장 정상화의 속도를 조율해야 한다고 생각했다 며 수요 규제를 조금 더 빠른 속도로 풀어나가서 시장이 안정 찾는데 최선의 노력할 것이라고 강조했다.',
 '이 같은 발언은 최근 고금리와 집값 하락 우려로 거래가 메마르면서 곳곳에서  경고음 이 터지는 데 따른 것으로 풀이된다.',
 '내놓은 매물이 나가지 않아 이사를 못가는 상황이 발생하거나,  영끌  매입자들의 금리 부담 급증, 집값 하락세에 따른 보유세 부담 호소, 빠른 월세화 현상 등이다.',
 '다주택자 세 중과 해제 검토